In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary:

In [2]:

# Calculate total schools in the district
total_schools = len(school_data_complete["school_name"].unique())

# Calculate total number of students in the district
total_students = school_data_complete["student_name"].count()

# Calculate total budget in the district
total_budget = school_data["budget"].sum()

# Calculate average Math score in the district
average_math = student_data["math_score"].mean()

# Calculate average Reading Score in the district
average_reading = student_data["reading_score"].mean()

# Calculate the percentage of students that passed math in the district
passdata_math = (student_data.loc[student_data["math_score"] >=70,"math_score"]).count()
passpercent_math = (passdata_math/total_students)*100

# Calculate the percentage of students that passed reading in the district.
passdata_reading = (student_data.loc[student_data["reading_score"] >=70,"reading_score"]).count()
passpercent_reading = (passdata_reading/total_students)*100

# Calculate the percentage of students that passed math **and** reading in the district.
pass_total = (student_data.loc[(student_data["reading_score"] >=70) & (student_data["math_score"] >=70) ,
                               "reading_score"]).count()
passpercent_overall = (pass_total/total_students)*100

# Creating a summary dataframe for entire district.
district_summary_df = pd.DataFrame({"Total Schools": [total_schools],
                           "Total Students": [total_students],
                           "Total Budget": [total_budget],
                           "Average Math Score":[average_math],
                           "Average Reading Score": [average_reading],
                           "% Passing Math": [passpercent_math],
                            "% Passing Reading": [passpercent_reading],
                            "% Overall Passing": [passpercent_overall]})

# Format the financial columns for better visual
district_summary_df["Total Students"] = district_summary_df["Total Students"].astype(float).map(
    "{:,.0f}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].astype(float).map(
    "${:,.2f}".format)

# Print the district summary output 
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary:

In [3]:

# Seggregate combined dataframe based on School name and generate columns: school type, no. of students,total budget,
# average math and reading scores by aggregae function 
school_names_df = school_data_complete.groupby(["school_name"]).agg({'type':pd.Series.mode,'student_name':['count'],
                                                                     'budget':pd.Series.mode,'math_score': ['mean'], 
                                                                   'reading_score': ['mean']})

# Rename these columns to desired text
school_names_df.columns = ['School Type', 'Total Students', 'Total School Budget', 'Average Math Score', 
                           'Average Reading Score' ]

# Calculate budget per student for every school and add it as a new column to the dataframe.
per_student_budget = school_names_df['Total School Budget']/school_names_df['Total Students']
school_names_df['Per Student Budget'] = per_student_budget

# Calculate passing percentage for Math for every school and add it as a new column to the dataframe.
pass_math = school_data_complete.groupby("school_name")['math_score'].apply(lambda x: (x>=70).sum())
school_names_df['% Passing Math'] = (pass_math/school_names_df['Total Students'])*100

# Calculate passing percentage for Reading for every school and add it as a new column to the dataframe.
pass_reading = school_data_complete.groupby("school_name")['reading_score'].apply(lambda x: (x>=70).sum())
school_names_df['% Passing Reading'] = (pass_reading/school_names_df['Total Students'])*100

# For Overall pass percentage based on both Math and Reading scores, first extract a dataframe from the merged dataframe
# consisting of data which matches both conditions.
math_reading = school_data_complete.loc[(school_data_complete["reading_score"] >=70) & 
                                        (school_data_complete["math_score"] >=70),:]

# Then groupby this new dataframe based on school name and preform a count function to get number of instances (=number of 
#  students who passed both in Math and Reading).
math_reading_sorted = math_reading.groupby(['school_name']) 
math_reading_sorted_df = math_reading_sorted.count()

# Calculate overall passing percent and add a new column with this data to the initial dataframe
school_names_df['% Overall Passing'] = (math_reading_sorted_df['Student ID']/school_names_df['Total Students'])*100

# Rearrange columns and assign it to a new dataframe which will represent the school summary data. 
school_summary_df = school_names_df[['School Type','Total Students','Total School Budget','Per Student Budget',
                                  'Average Math Score','Average Reading Score','% Passing Math','% Passing Reading',
                                   '% Overall Passing']]
# Format the financial columns for better visual
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].astype(float).map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].astype(float).map("${:,.2f}".format)

# Print the school summary output
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

In [4]:

# Arrange school summary data frame based on descending order of Overall Passing values and display top 5 values
top_schools_df = school_summary_df.sort_values('% Overall Passing', ascending=False)

# Print top 5 performing schools.
top_schools_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [5]:
# Arrange school summary data frame based on ascending order of Overall Passing values and display top 5 values
bottom_schools_df = school_summary_df.sort_values('% Overall Passing')

# Print bottom 5 performing schools.
bottom_schools_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [6]:
# Using student data original dataframe as it has all the necessary information for this output.
student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [7]:
# Check the values in grade column so that conditional statements for gradewise math score data can be derived
unique = student_data['grade'].unique()
unique

array(['9th', '12th', '11th', '10th'], dtype=object)

In [8]:
# Derive dataframe for 9th grade math data, group it by schools, and then apply mean function to math scores.
math_9th = student_data.loc[(student_data['grade']=='9th'),
                            ('math_score','school_name')]
grouped_math_9th = math_9th.groupby(['school_name'])
grouped_math_9th_df = pd.DataFrame(grouped_math_9th['math_score'].mean())

# Derive dataframe for 12th grade math data, group it by schools, and then apply mean function to math scores.
math_12th = student_data.loc[(student_data['grade']=='12th'),
                            ('math_score','school_name')]
grouped_math_12th = math_12th.groupby(['school_name'])
grouped_math_12th_df = pd.DataFrame(grouped_math_12th['math_score'].mean())

# Derive dataframe for 11th grade math data, group it by schools, and then apply mean function to math scores.
math_11th = student_data.loc[(student_data['grade']=='11th'),
                            ('math_score','school_name')]
grouped_math_11th = math_11th.groupby(['school_name'])
grouped_math_11th_df = pd.DataFrame(grouped_math_11th['math_score'].mean())

# Derive dataframe for 10th grade math data, group it by schools, and then apply mean function to math scores.
math_10th = student_data.loc[(student_data['grade']=='10th'),
                            ('math_score','school_name')]
grouped_math_10th = math_10th.groupby(['school_name'])
grouped_math_10th_df = pd.DataFrame(grouped_math_10th['math_score'].mean())

# combine 9th, 10th, 11th and 12th math average data into single dataframe for summarization 
summary_math_bygrade = pd.DataFrame({'9th': grouped_math_9th_df['math_score'],"10th": grouped_math_10th_df['math_score'],
                           "11th": grouped_math_11th_df['math_score'],"12th": grouped_math_12th_df['math_score']})
summary_math_bygrade.index.name = None 
# Print summary
summary_math_bygrade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Scores by Grade

In [9]:
# Derive dataframe for 9th grade redaing data, group it by schools, and then apply mean function to reading scores.
reading_9th = student_data.loc[(student_data['grade']=='9th'),
                            ('reading_score','school_name')]
grouped_reading_9th = reading_9th.groupby(['school_name'])
grouped_reading_9th_df = pd.DataFrame(grouped_reading_9th['reading_score'].mean())

# Derive dataframe for 12th grade reading data, group it by schools, and then apply mean function to reading scores.
reading_12th = student_data.loc[(student_data['grade']=='12th'),
                            ('reading_score','school_name')]
grouped_reading_12th = reading_12th.groupby(['school_name'])
grouped_reading_12th_df = pd.DataFrame(grouped_reading_12th['reading_score'].mean())

# Derive dataframe for 11th grade reading data, group it by schools, and then apply mean function to reading scores.
reading_11th = student_data.loc[(student_data['grade']=='11th'),
                            ('reading_score','school_name')]
grouped_reading_11th = reading_11th.groupby(['school_name'])
grouped_reading_11th_df = pd.DataFrame(grouped_reading_11th['reading_score'].mean())

# Derive dataframe for 10th grade reading data, group it by schools, and then apply mean function to reading scores.
reading_10th = student_data.loc[(student_data['grade']=='10th'),
                            ('reading_score','school_name')]
grouped_reading_10th = reading_10th.groupby(['school_name'])
grouped_reading_10th_df = pd.DataFrame(grouped_reading_10th['reading_score'].mean())

# combine 9th, 10th, 11th and 12th reading average data into single dataframe for summarization 
summary_reading_bygrade = pd.DataFrame({'9th': grouped_reading_9th_df['reading_score'],
                                        "10th": grouped_reading_10th_df['reading_score'],
                                        "11th": grouped_reading_11th_df['reading_score'],
                                        "12th": grouped_reading_12th_df['reading_score']})

summary_reading_bygrade.index.name = None
# Print summary
summary_reading_bygrade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending:

In [10]:
# Using unformatted school summary datframe to be able to filter based on numeric values of Per Student Budget column.
school_names_df

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,77.048432,81.033963,628.0,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,83.061895,83.975780,582.0,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,76.711767,81.158020,639.0,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,77.102592,80.746258,644.0,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,83.351499,83.816757,625.0,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,77.289752,80.934412,652.0,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,83.803279,83.814988,581.0,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,76.629414,81.182722,655.0,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,77.072464,80.966394,650.0,66.057551,81.222432,53.539172


In [11]:
# Determine max and min value in per student column to decide bin values
max = school_names_df["Per Student Budget"].max()
min = school_names_df["Per Student Budget"].min()
print(f"max= {max} , min= {min}")

max= 655.0 , min= 578.0


In [12]:
# Create bins in which to place vaues based on per student spending.
bins = [0, 585,630,645,676]

# Create labels for these bins
group_labels = ["<$585", "$585-629", "$630-644", "$645-675"]

# Slice the data and place it into bins and place the dataseries into a new column inside of the dataframe. 
school_names_df["Spending Ranges(Per Student)"] = pd.cut(school_names_df["Per Student Budget"], bins, labels = group_labels)

# Group this data based on this new column and perform required functions on respective columns. 
grouped_spending_range_df = school_names_df.groupby(["Spending Ranges(Per Student)"]).agg({'Average Math Score': ['mean'],
                                                'Average Reading Score': ['mean'],'% Passing Math':['mean'],
                                                '% Passing Reading':['mean'],'% Overall Passing':['mean'] })

# Rename the columns to remove 'mean' in column label.
grouped_spending_range_df.columns = ['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                                    '% Overall Passing']

#Round the column values to two decimal places.
grouped_spending_range_df= grouped_spending_range_df.round({'Average Math Score': 2, 'Average Reading Score': 2,
                                                            '% Passing Math': 2,'% Passing Reading': 2,
                                                            '% Overall Passing': 2 })
# Display summary table for scores by school spending                                                                                                 
grouped_spending_range_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges(Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size:

In [13]:
# Determine max and min value in total students column to decide bin values
max = school_names_df["Total Students"].max()
min = school_names_df["Total Students"].min()
print(f"max= {max} , min= {min}")

max= 4976 , min= 427


In [14]:
# Create bins in which to place vaues based on per student spending.
bins = [0, 1000,2000,5000]

# Create labels for these bins
group_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Slice the data and place it into bins and place the dataseries into a new column inside of the dataframe.
school_names_df["Total Students"] = pd.cut(school_names_df["Total Students"], bins, labels = group_labels)

# Group this data based on this new column and perform required functions on respective columns.
grouped_school_size_df = school_names_df.groupby(["Total Students"]).agg({'Average Math Score': ['mean'],
                                                'Average Reading Score': ['mean'],'% Passing Math':['mean'],
                                                '% Passing Reading':['mean'],'% Overall Passing':['mean']})

# Rename the columns to remove 'mean' in column label.
grouped_school_size_df.columns = ['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                                    '% Overall Passing']

#Round the column values to two decimal places.
grouped_school_size_df= grouped_school_size_df.round({'Average Math Score': 2, 'Average Reading Score': 2,'% Passing Math': 2,
                                 '% Passing Reading': 2,'% Overall Passing': 2 })

grouped_school_size_df.index.name = "School Size"
# Display summary table for scores by school size.
grouped_school_size_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

In [15]:
# Apply groupby function on school summary dataframe and apply appropriate functions to columns
grouped_school_type_df = school_names_df.groupby(["School Type"]).agg({'Average Math Score': ['mean'],
                                                'Average Reading Score': ['mean'],'% Passing Math':['mean'],
                                                '% Passing Reading':['mean'],'% Overall Passing':['mean'] })

# Rename the columns to remove 'mean' in column label.
grouped_school_type_df.columns = ['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                                    '% Overall Passing']

# Round the column values to two decimal places.                         
grouped_school_type_df= grouped_school_type_df.round({'Average Math Score': 2, 'Average Reading Score': 2,'% Passing Math': 2,
                                 '% Passing Reading': 2,'% Overall Passing': 2 })

grouped_school_type_df.index.name = None
# Print Summary table for scores by School Type.
grouped_school_type_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67


OBSERVATIONS:

1. Based on the money spent by schools per student, it appears that schools with a higher spending are actually performing worse than lower spending schools. 

2. Similarly, bigger the school in terms of size, poorer is the average performance in all categories. 

3. Overall, 'Charter' schools demonstrate a lot better performance when compared to 'District' schools. 